In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = False
is_export = False

landmarks_path = '../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

export_folder = 'output/rbf_ablation/'

# Data Loading

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=12, contour=contour, clip_bound='y', margin=0.5)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

# Ablation testing class

In [ ]:
from mesh4d.regist.reg_rbf import Obj4d_RBF, Trans_Nonrigid_RBF

class Obj4d_RBF_ablation(Obj4d_RBF):
    def __init__(self, enable_rigid: bool = False, enable_nonrigid: bool = False, k_nbr=1, kernel='thin_plate_spline', **kwargs):
        super().__init__(enable_rigid, enable_nonrigid, **kwargs)
        self.k_nbr = k_nbr
        self.kernel = kernel

    def process_nonrigid_dynamic(self, idx_source: int, idx_target: int, landmark_name: str, **kwargs):
        trans = Trans_Nonrigid_RBF(
            source_obj=self.obj_ls[idx_source],
            target_obj=self.obj_ls[idx_target],
        )
        trans.regist(landmark_name, k_nbr=self.k_nbr, kernel=self.kernel, **kwargs)
        self.obj_ls[idx_source].set_trans_nonrigid(trans)

# Kernel selection

In [ ]:
kernel_results = {
    'control landmarks': {},
    'non-control landmarks': {},
}

## Control landmarks

In [ ]:
import time
import mesh4d
from mesh4d import kps
mesh4d.output_msg = False

for k_nbr in [0, ]:
    for kernel in ['thin_plate_spline', 'linear', 'cubic', 'quintic']:
        kernel_results['control landmarks'][kernel] = {}
        results = kernel_results['control landmarks'][kernel]
        
        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        # registration
        start_time = time.time()

        o4 = Obj4d_RBF_ablation(
            fps=120 / stride,
            enable_rigid=False,
            enable_nonrigid=True,
            k_nbr=k_nbr,
            kernel=kernel,
        )

        o4.add_obj(*body_ls)
        o4.load_markerset('landmarks', landmarks)
        o4.regist('landmarks')

        duration = time.time() - start_time
        
        # computation time
        print("computation time: {}".format(duration))
        results['duration'] = duration
        
        # control landmarks
        kps_source = landmarks.get_time_coord(0)
        o4.vkps_track(kps_source, frame_id=0)
        vkps = o4.assemble_markerset(name='vkps')
        diff = kps.MarkerSet.diff(vkps, landmarks)

        print(diff['diff_str'])
        results['dist_mean'] = diff['dist_mean']
        results['dist_std'] = diff['dist_std']

## Non-control landmarks

In [ ]:
import mesh4d
import numpy as np
mesh4d.output_msg = False

for k_nbr in [0, ]:
    for kernel in ['thin_plate_spline', 'linear', 'cubic', 'quintic']:
        kernel_results['non-control landmarks'][kernel] = {}
        results = kernel_results['non-control landmarks'][kernel]

        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        dist_ls = []
        duration_ls = []

        # k-fold cross-verification
        for name in landmarks.markers.keys():
            # split dataset
            landmarks_test, landmarks_train = landmarks.split((name, ))

            # registration
            start_time = time.time()
            
            o4 = Obj4d_RBF_ablation(
                fps=120 / stride,
                enable_rigid=False,
                enable_nonrigid=True,
                k_nbr=k_nbr,
                kernel=kernel,
            )

            o4.add_obj(*body_ls)
            o4.load_markerset('landmarks_train', landmarks_train)
            o4.load_markerset('landmarks_test', landmarks_test)
            o4.regist('landmarks_train')

            duration = time.time() - start_time
            duration_ls.append(duration)

            # virtual key points tracking
            kps_source = landmarks_test.get_time_coord(0)
            o4.vkps_track(kps_source, frame_id=0, name='vkps')
            vkps = o4.assemble_markerset(name='vkps')
            diff = kps.MarkerSet.diff(vkps, landmarks_test)

            for marker_diff in diff['diff_dict'].values():
                dist_ls.append(marker_diff['dist'])
        
        dist_mean = np.mean(np.array(dist_ls))
        dist_std = np.std(np.array(dist_ls))

        print("computation time: {}".format(np.mean(np.array(duration))))
        print('overall error: {:.2f} ± {:.2f} (mm)'.format(dist_mean, dist_std))

        results['duration'] = duration
        results['dist_mean'] = diff['dist_mean']
        results['dist_std'] = diff['dist_std']

## Visualization

In [ ]:
# data aggregate
duration_ls = []
control_dist_mean_ls = []
control_dist_std_ls = []

for kernel in kernel_results['control landmarks'].values():
    duration_ls.append(kernel['duration'])
    control_dist_mean_ls.append(kernel['dist_mean']/10)
    control_dist_std_ls.append(kernel['dist_std']/10)

noncontrol_dist_mean_ls = []
noncontrol_dist_std_ls = []

for kernel in kernel_results['non-control landmarks'].values():
    noncontrol_dist_mean_ls.append(kernel['dist_mean']/10)
    noncontrol_dist_std_ls.append(kernel['dist_std']/10)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

# control landmarks
ax1.bar(np.arange(
    len(control_dist_mean_ls)) * 4, control_dist_mean_ls, color='olive',
    label='control landmarks alginment',
    tick_label= ['TPS', 'linear', 'cubic', 'quintic'],
    )

# non-control landmarks
ax1.bar(
    np.arange(len(noncontrol_dist_mean_ls)) * 4 + 1, noncontrol_dist_mean_ls, color='goldenrod',
    label='non-control landmarks alginment',
    )

# duration
ax2.bar(np.arange(len(duration_ls)) * 4 + 2, duration_ls, color='teal', label='computation time')

ax1.set_ylabel('Alignment Error (cm)')
ax2.set_ylabel('Computation Time (s)')
ax1.set_yscale('log')
ax1.legend(loc=[0.25, 0.85])
ax2.legend(loc=[0.25, 0.77])
plt.grid(linestyle='--', axis='y')

# Post-alignment

In [ ]:
align_results = {
    'control landmarks': {},
    'non-control landmarks': {},
}

## Shape mergence improvement

In [ ]:
import mesh4d
import numpy as np
from scipy.spatial import KDTree
mesh4d.output_msg = False

for k_nbr in [0, 1, 2, 5]:
    for kernel in ['thin_plate_spline', ]:
        align_results['merge'][k_nbr] = {}
        results = align_results['merge'][k_nbr]

        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        # registration
        start_time = time.time()

        o4 = Obj4d_RBF_ablation(
            fps=120 / stride,
            enable_rigid=False,
            enable_nonrigid=True,
            k_nbr=k_nbr,
            kernel=kernel,
        )

        o4.add_obj(*body_ls)
        o4.load_markerset('landmarks', landmarks)
        o4.regist('landmarks')

        duration = time.time() - start_time
        
        # computation time
        print("computation time: {}".format(duration))
        results['duration'] = duration
        
        # shape mergence
        dist_ls = []

        for id in range(len(o4.obj_ls) - 1):
            # get predicted deform points
            source_points = o4.obj_ls[id].get_vertices()
            deform_points = o4.obj_ls[id].trans_nonrigid.shift_points(source_points)
            
            # search deform points' nearest points in target points
            target_points = o4.obj_ls[id + 1].get_vertices()
            tree = KDTree(target_points)
            _, idx = tree.query(deform_points)
            target_nearest_points = target_points[idx]

            # calculate the distance from deform points and its nearest points in target points
            dist = np.linalg.norm(target_nearest_points - deform_points, axis=1)
            dist_ls.append(dist)

        dist_array = np.concatenate(dist_ls)
        results['dist_mean'] = np.mean(dist_array)
        results['dist_std'] = np.std(dist_array)

        print("nearest point distance: {:.2f} ± {:.2f} (mm)".format(results['dist_mean'], results['dist_std']))

## Non-control landmarks tracking

In [ ]:
import mesh4d
import numpy as np
mesh4d.output_msg = False

for k_nbr in [0, 1, 2, 5]:
    for kernel in ['thin_plate_spline', ]:
        align_results['non-control landmarks'][k_nbr] = {}
        results = align_results['non-control landmarks'][k_nbr]

        print('='*70)
        print('knbr {} - kernel {}'.format(k_nbr, kernel))

        dist_ls = []
        duration_ls = []

        # k-fold cross-verification
        for name in landmarks.markers.keys():
            # split dataset
            landmarks_test, landmarks_train = landmarks.split((name, ))

            # registration
            start_time = time.time()
            
            o4 = Obj4d_RBF_ablation(
                fps=120 / stride,
                enable_rigid=False,
                enable_nonrigid=True,
                k_nbr=k_nbr,
                kernel=kernel,
            )

            o4.add_obj(*body_ls)
            o4.load_markerset('landmarks_train', landmarks_train)
            o4.load_markerset('landmarks_test', landmarks_test)
            o4.regist('landmarks_train')

            duration = time.time() - start_time
            duration_ls.append(duration)

            # virtual key points tracking
            kps_source = landmarks_test.get_time_coord(0)
            o4.vkps_track(kps_source, frame_id=0, name='vkps')
            vkps = o4.assemble_markerset(name='vkps')
            diff = kps.MarkerSet.diff(vkps, landmarks_test)

            for marker_diff in diff['diff_dict'].values():
                dist_ls.append(marker_diff['dist'])
        
        dist_mean = np.mean(np.array(dist_ls))
        dist_std = np.std(np.array(dist_ls))

        print("computation time: {}".format(np.mean(np.array(duration))))
        print('overall error: {:.2f} ± {:.2f} (mm)'.format(dist_mean, dist_std))

        results['duration'] = duration
        results['dist_mean'] = diff['dist_mean']
        results['dist_std'] = diff['dist_std']

In [ ]:
align_results['non-control landmarks']

## Visualization

In [ ]:
# data aggregate
duration_ls = []
merge_dist_mean_ls = []
noncontrol_dist_mean_ls = []

for k_nbr in align_results['merge'].values():
    duration_ls.append(k_nbr['duration'])
    merge_dist_mean_ls.append(k_nbr['dist_mean']/10)

for k_nbr in align_results['merge'].values():
    noncontrol_dist_mean_ls.append(k_nbr['dist_mean']/10)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

# shape mergence
ax1.bar(np.arange(
    len(merge_dist_mean_ls)) * 4, merge_dist_mean_ls, color='olive',
    label='shape alignment',
    )

# non-control landmarks
ax1.bar(np.arange(
    len(noncontrol_dist_mean_ls)) * 4 + 1, merge_dist_mean_ls, color='goldenrod',
    label='non-control landmarks',
    tick_label= ['None', '1', '2', '5'],
    )

# duration
ax2.bar(np.arange(len(duration_ls)) * 4 + 2, duration_ls, color='teal', label='computation time')

ax1.set_xlabel('$M$ in post alignment ')
ax1.set_ylabel('Shape Alignment Distance (cm)')
ax1.set_yscale('log')
ax1.legend(loc=[0.01, 0.48])

ax2.set_ylabel('Computation Time (s)')
ax2.legend(loc=[0.01, 0.4])

plt.grid(linestyle='--', axis='y')

## Tracking accuracy improvement

from numpy import random
from numpy.random import default_rng

g = default_rng(42)

align_results = {}
try_num = 3

for point_num in [4, 8, 12, 16]:
    align_results[point_num] = {}
        
    for k_nbr in [0, 1]:
        for kernel in ['thin_plate_spline', ]:
            print('='*70)
            print('point_num {} - knbr {} - kernel {}'.format(point_num, k_nbr, kernel))
            
            align_results[point_num][k_nbr] = {}
            results = align_results[point_num][k_nbr]
            dist_ls = []
            duration_ls = []
            
            for _ in range(try_num):
                # split dataset
                indices = g.choice(18, size=point_num, replace=False)
                landmarks_train, landmarks_test = landmarks.split(['marker {}'.format(id) for id in indices])
                
                # registration
                start_time = time.time()
                
                o4 = Obj4d_RBF_ablation(
                    fps=120 / stride,
                    enable_rigid=False,
                    enable_nonrigid=True,
                    k_nbr=k_nbr,
                    kernel=kernel,
                )

                o4.add_obj(*body_ls)
                o4.load_markerset('landmarks_train', landmarks_train)
                o4.load_markerset('landmarks_test', landmarks_test)
                o4.regist('landmarks_train')

                duration = time.time() - start_time
                duration_ls.append(duration)

                # virtual key points tracking
                kps_source = landmarks_test.get_time_coord(0)
                o4.vkps_track(kps_source, frame_id=0, name='vkps')
                vkps = o4.assemble_markerset(name='vkps')
                diff = kps.MarkerSet.diff(vkps, landmarks_test)

                for marker_diff in diff['diff_dict'].values():
                    dist_ls.append(marker_diff['dist'])
                
            dist_mean = np.mean(np.array(dist_ls))
            dist_std = np.std(np.array(dist_ls))
            results['dist_mean'] = dist_mean
            results['dist_std'] = dist_std

            print('-'*70)
            print("computation time: {}".format(np.mean(np.array(duration))))
            print('overall error: {:.2f} ± {:.2f} (mm)'.format(dist_mean, dist_std))

from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600

# sample improvements
ratio_ls = []
improve_ls = []

for point_num, result in align_results.items():
    ratio_ls.append(point_num / 18 * 100)
    improve_ls.append((1 - result[1]['dist_mean'] / result[0]['dist_mean']) * 100)

plt.scatter(ratio_ls, improve_ls)

# continues improvements
y = interp1d(ratio_ls, improve_ls, kind='quadratic')
xs = np.linspace(np.min(ratio_ls), np.max(ratio_ls), num=100, endpoint=True)
plt.plot(xs, y(xs), linestyle='--')

plt.xlabel('Landmarks Coverage Rate (%)')
plt.ylabel('Accuracy Improvement (%)')
plt.grid('--')